In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import yfinance as yf
import quandl
import statsmodels.api as sm
import seaborn as sns
import scipy.stats as stats
import datetime as dt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
apikey = 'J_fXGeVW_zC6RaDeJSQv'
quandl.ApiConfig.api_key = apikey

In [54]:
d = dt.datetime.today() - dt.timedelta(days= 1)


In [55]:
str(d)

'2023-04-05 02:29:07.848083'

In [111]:
class Portfolio:

    def __init__(self,tickers:list(), shares:list(),cash:float):
        self.tickers = tickers
        self.shares = shares
        self.cash = cash



    def return_tick(self):
        return self.tickers
    def fetch_price_data(self, enddate = None):
        if enddate == None:
            enddate = dt.datetime.today() - dt.timedelta(days= 1)
        else:
            enddate = enddate
        start_date = '2015-01-01'
        price_df = quandl.get_table('QUOTEMEDIA/PRICES', ticker =self.tickers,\
                          qopts = {'columns':['date','ticker','adj_close']},paginate = True, date = {'gte':start_date,'lte':enddate}).set_index('date')
        return price_df.pivot_table(values = 'adj_close', columns='ticker', index=price_df.index)
        



    def portfolio_(self):
        price_data = self.fetch_price_data()
        lp = price_data.iloc[-1][self.tickers].to_frame("Spot_Price")
        P = pd.DataFrame({'ticker':self.tickers,'Shares':self.shares})
        P = P.merge(lp, on = 'ticker')
        P['MV'] = P['Shares']*P['Spot_Price']
        cash_row = pd.DataFrame({'ticker':['Cash'], 'Shares':[0],'Spot_Price':[1],'MV':[self.cash]})
        
        total_row = pd.DataFrame({'ticker': ['TOTAL'], 'Shares': [0], 'Spot_Price': [0], 'MV': [P['MV'].sum()]})
        
        tot_val = P['MV'].sum()
        
        P = pd.concat([P,cash_row], axis = 0)
        P = pd.concat([P,total_row], axis=0)
        P['Weight'] = P['MV']/(tot_val)
        P 
        

        return np.round(P,5)    
    
    def covariance_matrix(self):
        price_data = self.fetch_price_data()
        rets = price_data.pct_change().dropna()
        cov = rets.cov()
        return cov
    def expected_return(self):
        price_data = self.fetch_price_data()
        rets = price_data.pct_change().dropna()
        return rets.mean()
        
    def expected_peformance(self, time_factor = 1):
        cov_matrix = self.covariance_matrix()
        portfolio = self.portfolio_()
        MVs_port = portfolio.iloc[:-2]['MV'].values
        weights = portfolio.iloc[:-2]['Weight'].values
        
        mu = self.expected_return().values
        expect_mu_p = weights.T @ mu
        dollar_mu = MVs_port.T @ mu
        sigma =  np.sqrt(MVs_port.T @ cov_matrix @ MVs_port)
        sigma_ret = np.sqrt(weights.T @ cov_matrix @ weights)
        expected_peformance = pd.DataFrame(data = None, index=['Expected_Return','Expected_Sigma','$Expected_MU','$Expected_Sigma'], columns=['Expected_Performance'])
        
        expected_peformance.loc['Expected_Return','Expected_Performance'] = expect_mu_p*time_factor
        expected_peformance.loc['Expected_Sigma','Expected_Performance'] = sigma_ret*np.sqrt(time_factor)

        expected_peformance.loc['$Expected_MU','Expected_Performance'] = dollar_mu*time_factor

        expected_peformance.loc['$Expected_Sigma','Expected_Performance'] = sigma *np.sqrt(time_factor)

        return expected_peformance
        
        

    
    
        
                

        


    

In [241]:
p = Portfolio(['JPM','AAPL'],[20,80], 5000)
t = p.return_tick()

In [242]:
PVs = p.fetch_price_data()

In [243]:
PVs.index[-1]

Timestamp('2023-04-05 00:00:00')

In [205]:
np.zeros(2)

array([0., 0.])

In [66]:

my_ticks = ['AMZN',"COST","HD","JEPI","VTV",'WAL']
num_shares = [6,4.00346+.00712,7.05887,10,4.02437,25]

In [112]:
myPort = Portfolio(tickers=my_ticks,shares = num_shares,cash = 5432.06)


In [82]:
tick_data = myPort.fetch_price_data()

In [114]:
portfolio = myPort.portfolio_()
portfolio

,ticker,Shares,Spot_Price,MV,Weight
0,AMZN,6.00000,101.10,606.60000,0.09370
1,COST,4.01058,497.13,1993.77964,0.30797
2,HD,7.05887,288.67,2037.68400,0.31475
3,JEPI,10.00000,54.30,543.00000,0.08387
4,VTV,4.02437,138.83,558.70329,0.08630
5,WAL,25.00000,29.37,734.25000,0.11341
0,Cash,0.00000,1.00,5432.06000,0.83906
0,TOTAL,0.00000,0.00,6474.01693,1.00000


In [113]:
myPort.expected_peformance(252)

,Expected_Performance
Expected_Return,0.147113
Expected_Sigma,0.208352
$Expected_MU,952.412103
$Expected_Sigma,1348.878345


In [106]:
myPort.expected_return().mean()

0.0005122777903916977

,ticker,Shares,Spot_Price,MV
0,AMZN,6.00000,101.10,606.60000
1,COST,4.01058,497.13,1993.77964
2,HD,7.05887,288.67,2037.68400
3,JEPI,10.00000,54.30,543.00000
4,VTV,4.02437,138.83,558.70329
5,WAL,25.00000,29.37,734.25000


In [38]:
cov = myPort.covariance_matrix()

In [39]:
cov

ticker,AMZN,COST,HD,JEPI,VTV,WAL
ticker,,,,,,
AMZN,0.000583,0.000189,0.000189,0.000097,0.000107,0.000206
COST,0.000189,0.000232,0.000141,0.000072,0.000077,0.000104
HD,0.000189,0.000141,0.000257,0.000075,0.000096,0.000162
JEPI,0.000097,0.000072,0.000075,0.000054,0.000065,0.000106
VTV,0.000107,0.000077,0.000096,0.000065,0.000112,0.000235
WAL,0.000206,0.000104,0.000162,0.000106,0.000235,0.001396
